In [39]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

## Task1

빈 칸을 채워주세요!

단계별 output이 github 파일에는 남아있으니 그 output과 동일한 형태인지 확인하면서 진행해주시면 됩니다~

In [40]:
#1. 생성할 문장 데이터

sentence = ("Brick walls are there for a reason and you must not think "
            "that the brick walls aren't there to keep us out, but rather "
            "in this way that the brick walls are there to show us how badly we want things.")

In [41]:
sentence         # 한 문장으로 구성됨

"Brick walls are there for a reason and you must not think that the brick walls aren't there to keep us out, but rather in this way that the brick walls are there to show us how badly we want things."

In [42]:
#2. 문자 집합 만들기
word_set = list(set(sentence))

## 문제(1): 각 문자에 정수 인코딩 (공백도 하나의 원소로 포함)
vocab = {word: idx for idx, word in enumerate(word_set)}

In [43]:
print(vocab)

{',': 0, 'f': 1, 'l': 2, 'k': 3, 'r': 4, '.': 5, 'h': 6, ' ': 7, 'm': 8, 'p': 9, 'n': 10, 'g': 11, 'w': 12, 'o': 13, 'b': 14, 't': 15, "'": 16, 'B': 17, 'y': 18, 'e': 19, 'u': 20, 'i': 21, 'c': 22, 'd': 23, 'a': 24, 's': 25}


In [44]:
#3. 문자 집합 크기 확인

vocab_size = len(vocab)
print('문자 집합 크기 : {}'.format(vocab_size))    # 총 26가지의 문자가 사용됨!

문자 집합 크기 : 26


In [45]:
#4. 하이퍼 파라미터 설정(자유롭게 수정해보세요!)

hidden_size = vocab_size # 같아야 하는 것 확인!
sequence_length = 10  # 너무 길거나 너무 짧게 잡으면 안됩니다!
learning_rate = 0.01

In [46]:
sentence[:10]

'Brick wall'

In [47]:
sentence[:sequence_length]

'Brick wall'

In [48]:
#5. seqence 길이 단위 자르기

# 데이터 구성을 위한 리스트
x_data = []
y_data = []

## 문제(2): 반복문 내에서의 인덱싱을 사용하여 sequence_length 값 단위로 샘플을 잘라 데이터 만들기, y_str은 x_str은 한 칸씩 쉬프트된 sequnce

for i in range(0, len(sentence) - sequence_length):
  x_str = sentence[i:i+sequence_length]
  y_str = sentence[i+1:i+1+sequence_length]
  print(i, x_str, "->", y_str)

  # x_str과 y_str이 문자집합에 해당하는 인덱스를 각각 x_data, y_data에 append
  x_data.append([vocab[c] for c in x_str])
  y_data.append([vocab[d] for d in y_str])

0 Brick wall -> rick walls
1 rick walls -> ick walls 
2 ick walls  -> ck walls a
3 ck walls a -> k walls ar
4 k walls ar ->  walls are
5  walls are -> walls are 
6 walls are  -> alls are t
7 alls are t -> lls are th
8 lls are th -> ls are the
9 ls are the -> s are ther
10 s are ther ->  are there
11  are there -> are there 
12 are there  -> re there f
13 re there f -> e there fo
14 e there fo ->  there for
15  there for -> there for 
16 there for  -> here for a
17 here for a -> ere for a 
18 ere for a  -> re for a r
19 re for a r -> e for a re
20 e for a re ->  for a rea
21  for a rea -> for a reas
22 for a reas -> or a reaso
23 or a reaso -> r a reason
24 r a reason ->  a reason 
25  a reason  -> a reason a
26 a reason a ->  reason an
27  reason an -> reason and
28 reason and -> eason and 
29 eason and  -> ason and y
30 ason and y -> son and yo
31 son and yo -> on and you
32 on and you -> n and you 
33 n and you  ->  and you m
34  and you m -> and you mu
35 and you mu -> nd you mus
36

In [49]:
# 출력해서 한 칸씩 쉬프트된 것 확인하기!

print(x_data[0])
print(y_data[0])     # x_data와 비교했을 때 한칸씩 밀려있음!

[17, 4, 21, 22, 3, 7, 12, 24, 2, 2]
[4, 21, 22, 3, 7, 12, 24, 2, 2, 25]


#### np.eye란?
- 단위행렬을 생성하는 함수
- np.eye(N=행 수, M=열 수(Default=None, 이 경우 N과 같은 값 즉 정사각 행렬 생성))

In [50]:
[np.eye(vocab_size)[i] for i in x_data]      # 각 word가 feature가 되므로 vocab size가 인수로 들어가야함!

[array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0.,

In [51]:
for i in x_data[0]:
  print(i)

17
4
21
22
3
7
12
24
2
2


In [52]:
[np.eye(vocab_size)[i] for i in x_data[0]]     # 순서대로 17, 4, 21, 22 ... 에 1이 입력되어있음

[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        1., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 1., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 0.]),
 array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 1., 0.]),
 array([0., 0., 1., 0., 

In [53]:
##6. 입력 시퀀스에 대해 원핫인코딩 수행

## 문제(4) : x_data를 원핫인코딩 > numpy의 eye를 쓸 수 있지 않을까?
x_one_hot = [np.eye(vocab_size)[i] for i in x_data]

##7. 입력 데이터, 레이블데이터 텐서로 변환

## 문제(5) : x_one_hot과 y_data 텐서로 변환 : 둘 다 같은 형식의 텐서로 변환하면 될까?? (FloatTensor, LongTesor 중 맞는 것은?)
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

In [54]:
y_data[0]    # x_one_hot은 넘파이 배열, y_data는 정수 배열이므로 각각 FloatTensor, LongTensor로 변환해주어야 함!

[4, 21, 22, 3, 7, 12, 24, 2, 2, 25]

In [55]:
##8. 크기 확인
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([188, 10, 26])
레이블의 크기 : torch.Size([188, 10])


In [56]:
##9.원핫인코딩 결과 샘플 확인하기
print(X[0])

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0.,

In [57]:
##10. 레이블 데이터 샘플 확인하기
print(Y[0])

tensor([ 4, 21, 22,  3,  7, 12, 24,  2,  2, 25])


In [58]:
##11. RNN 모델 구현

##문제(6) : 기본 pytorch 인자 넣기 연습 + forward 채우기
### 조건 : rnn layer 2개 쌓기 + 마지막은 fc layer
### batch_fisrt 설정 필요할까? (유튜브 강의 참고)

class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_size=input_dim, hidden_size=hidden_dim, num_layers=layers, batch_first=True)
    self.fc = torch.nn.Linear(hidden_dim, input_dim)

  def forward(self, x):
    x, _=self.rnn(x)
    x=self.fc(x)
    return x

RNN의 파라미터들: https://pytorch.org/docs/stable/generated/torch.nn.RNN.html

In [59]:
net = Net(vocab_size, hidden_size, 2)

In [60]:
##12. loss function
criterion = torch.nn.CrossEntropyLoss()

##13. optimizer
optimizer = optim.Adam(net.parameters(), learning_rate)

In [61]:
##14. 출력 크기 점검
outputs = net(X)
print(outputs.shape)

torch.Size([188, 10, 26])


In [62]:
##15. Training 시작

for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    ##문제(7) : outputs, Y 형태 그대로 넣으면 안되죠. view 함수를 이용해 loss값을 계산해봅시다.
    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([word_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += word_set[result[-1]]

    print(predict_str)

uuccBccBBccucucucBuucucBuuuuuccc'uucuucuc'uu'''uuuuuuuu'ccBBuBuuuuucuccBcccBccucucuccuuuuucuccuucnnc'cu'''uuc''uucucuucucucBucucBucBuu'uuuuucuccBcccBccucucucBuucuccuu'''u'cu'''uuu''cuccuccucuuBucuc
tnttttcttcttcnctcttnctctntntnttttanctatntttntaannttnntnantttttnnttnntntttcttcttcnctncttttnctctnnttncntntnttnnttnnttttntntnctncnctnctttannttnntntttcttcttcnctcttnctctncttanctnttcnntnattctncttctttntac
ttttttttttttttttattattatatttattaaatttatatttttaatatttattatttttattttttattttttttttttttatttttattattttttttatttttttttttatttatttttttatatattttattttttattttttttttttttattattattattatattttatttaattattataattttaat
ttttttttttttttttttttttttttttttttttttttttttttttattttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttatttttttttttttttttttttttttttttttttttttttatttttttttttttttttattttttttttttt
ttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttt
tttttttttt

In [63]:
predict_str   # BASE: sequence length=10, learning rate=0.01

"rick walls are there tor a reason and you must not think that the brick walls are 't there to keep us out, but rather in this way that the brick walls are there to khow us how badly we want thinksk"

In [72]:
# 파라미터 조정(1): sequence length=10, learning rate=0.05

hidden_size = vocab_size
sequence_length = 10
learning_rate = 0.05    # learning rate 높여보기

x_data = []
y_data = []

for i in range(0, len(sentence) - sequence_length):
  x_str = sentence[i:i+sequence_length]
  y_str = sentence[i+1:i+1+sequence_length]
  print(i, x_str, "->", y_str)

  x_data.append([vocab[c] for c in x_str])
  y_data.append([vocab[d] for d in y_str])

x_one_hot = [np.eye(vocab_size)[i] for i in x_data]

X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_size=input_dim, hidden_size=hidden_dim, num_layers=layers, batch_first=True)
    self.fc = torch.nn.Linear(hidden_dim, input_dim)

  def forward(self, x):
    x, _=self.rnn(x)
    x=self.fc(x)
    return x

net = Net(vocab_size, hidden_size, 2)
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)
outputs = net(X)

for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0:
            predict_str += ''.join([word_set[t] for t in result])
        else:
            predict_str += word_set[result[-1]]

    print(predict_str)

0 Brick wall -> rick walls
1 rick walls -> ick walls 
2 ick walls  -> ck walls a
3 ck walls a -> k walls ar
4 k walls ar ->  walls are
5  walls are -> walls are 
6 walls are  -> alls are t
7 alls are t -> lls are th
8 lls are th -> ls are the
9 ls are the -> s are ther
10 s are ther ->  are there
11  are there -> are there 
12 are there  -> re there f
13 re there f -> e there fo
14 e there fo ->  there for
15  there for -> there for 
16 there for  -> here for a
17 here for a -> ere for a 
18 ere for a  -> re for a r
19 re for a r -> e for a re
20 e for a re ->  for a rea
21  for a rea -> for a reas
22 for a reas -> or a reaso
23 or a reaso -> r a reason
24 r a reason ->  a reason 
25  a reason  -> a reason a
26 a reason a ->  reason an
27  reason an -> reason and
28 reason and -> eason and 
29 eason and  -> ason and y
30 ason and y -> son and yo
31 son and yo -> on and you
32 on and you -> n and you 
33 n and you  ->  and you m
34  and you m -> and you mu
35 and you mu -> nd you mus
36

In [73]:
predict_str    # sequence length=10, learning rate=0.05

"rick walls are there for a reason and you must not think that the brick walls are 't there to keep us out, but rather in this way that the brick walls are there fo khow us how badly we want things."

In [74]:
# 파라미터 조정(2): sequence length=7, learning rate=0.01

hidden_size = vocab_size
sequence_length = 7  # sequence length 줄여보기
learning_rate = 0.01

x_data = []
y_data = []

for i in range(0, len(sentence) - sequence_length):
  x_str = sentence[i:i+sequence_length]
  y_str = sentence[i+1:i+1+sequence_length]
  print(i, x_str, "->", y_str)

  x_data.append([vocab[c] for c in x_str])
  y_data.append([vocab[d] for d in y_str])

x_one_hot = [np.eye(vocab_size)[i] for i in x_data]

X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_size=input_dim, hidden_size=hidden_dim, num_layers=layers, batch_first=True)
    self.fc = torch.nn.Linear(hidden_dim, input_dim)

  def forward(self, x):
    x, _=self.rnn(x)
    x=self.fc(x)
    return x

net = Net(vocab_size, hidden_size, 2)
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)
outputs = net(X)

for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0:
            predict_str += ''.join([word_set[t] for t in result])
        else:
            predict_str += word_set[result[-1]]

    print(predict_str)

0 Brick w -> rick wa
1 rick wa -> ick wal
2 ick wal -> ck wall
3 ck wall -> k walls
4 k walls ->  walls 
5  walls  -> walls a
6 walls a -> alls ar
7 alls ar -> lls are
8 lls are -> ls are 
9 ls are  -> s are t
10 s are t ->  are th
11  are th -> are the
12 are the -> re ther
13 re ther -> e there
14 e there ->  there 
15  there  -> there f
16 there f -> here fo
17 here fo -> ere for
18 ere for -> re for 
19 re for  -> e for a
20 e for a ->  for a 
21  for a  -> for a r
22 for a r -> or a re
23 or a re -> r a rea
24 r a rea ->  a reas
25  a reas -> a reaso
26 a reaso ->  reason
27  reason -> reason 
28 reason  -> eason a
29 eason a -> ason an
30 ason an -> son and
31 son and -> on and 
32 on and  -> n and y
33 n and y ->  and yo
34  and yo -> and you
35 and you -> nd you 
36 nd you  -> d you m
37 d you m ->  you mu
38  you mu -> you mus
39 you mus -> ou must
40 ou must -> u must 
41 u must  ->  must n
42  must n -> must no
43 must no -> ust not
44 ust not -> st not 
45 st not  -> t not 

In [75]:
predict_str    # sequence length=7, learning rate=0.01

"rick walls are there tor a reason and you must not think that the brick walls are 't there to seep us out, but rather in this way that the brick walls are there to show us how badly we want thinks."

In [76]:
# 파라미터 조정(3): sequence length=12, learning rate=0.01

hidden_size = vocab_size
sequence_length = 12  # sequence length 높여보기
learning_rate = 0.01

x_data = []
y_data = []

for i in range(0, len(sentence) - sequence_length):
  x_str = sentence[i:i+sequence_length]
  y_str = sentence[i+1:i+1+sequence_length]
  print(i, x_str, "->", y_str)

  x_data.append([vocab[c] for c in x_str])
  y_data.append([vocab[d] for d in y_str])

x_one_hot = [np.eye(vocab_size)[i] for i in x_data]

X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_size=input_dim, hidden_size=hidden_dim, num_layers=layers, batch_first=True)
    self.fc = torch.nn.Linear(hidden_dim, input_dim)

  def forward(self, x):
    x, _=self.rnn(x)
    x=self.fc(x)
    return x

net = Net(vocab_size, hidden_size, 2)
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)
outputs = net(X)

for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0:
            predict_str += ''.join([word_set[t] for t in result])
        else:
            predict_str += word_set[result[-1]]

    print(predict_str)

0 Brick walls  -> rick walls a
1 rick walls a -> ick walls ar
2 ick walls ar -> ck walls are
3 ck walls are -> k walls are 
4 k walls are  ->  walls are t
5  walls are t -> walls are th
6 walls are th -> alls are the
7 alls are the -> lls are ther
8 lls are ther -> ls are there
9 ls are there -> s are there 
10 s are there  ->  are there f
11  are there f -> are there fo
12 are there fo -> re there for
13 re there for -> e there for 
14 e there for  ->  there for a
15  there for a -> there for a 
16 there for a  -> here for a r
17 here for a r -> ere for a re
18 ere for a re -> re for a rea
19 re for a rea -> e for a reas
20 e for a reas ->  for a reaso
21  for a reaso -> for a reason
22 for a reason -> or a reason 
23 or a reason  -> r a reason a
24 r a reason a ->  a reason an
25  a reason an -> a reason and
26 a reason and ->  reason and 
27  reason and  -> reason and y
28 reason and y -> eason and yo
29 eason and yo -> ason and you
30 ason and you -> son and you 
31 son and you  ->

In [77]:
predict_str    # sequence length=12, learning rate=0.01

"rick walls are there tor a reason and you must not think that the brick walls are 't there to keep us out, but rather in this way that the brick walls are there to khow us how badly we want thinksd"

결과가 어떤가요?? 마지막 에폭의 문장이 그럴싸한가요?

BASE: sequence_length=10, learning_rate=0.01
  - 'rick walls are there tor a reason and you must not think that the brick walls are 't there to keep us out, but rather in this way that the brick walls are there to khow us how badly we want thinksk'
  - Brick의 B 생략, for 대신 tor, aren't의 n 생략 등의 오류 존재
  - 오류를 감안하고 본다면 그럴듯한 문장 생성

(1): sequence_length=10, learning_rate=0.05
  - 'rick walls are there for a reason and you must not think that the brick walls are 't there to keep us out, but rather in this way that the brick walls are there fo khow us how badly we want things.'
  - 베이스가 잡지 못한 오류 tor->for, to->for, thinksk->things 등을 잡아냄
  - 여전히 Brick의 B 생략, aren't의 n 생략 등의 오류는 발생

(2): sequence_length=7, learning_rate=0.01
  -  'rick walls are there tor a reason and you must not think that the brick walls are 't there to seep us out, but rather in this way that the brick walls are there to show us how badly we want thinks.'
  - 베이스에서 발생한 오류가 그대로 나타남
  - 베이스에서 발생하지 않았던 오류(keep->seep)나 베이스의 오류를 잡아내기도(khow->show, thinksk->thinks) 함

(3): sequence_length=12, learning_rate=0.01
  -  'rick walls are there tor a reason and you must not think that the brick walls are 't there to keep us out, but rather in this way that the brick walls are there to khow us how badly we want thinksd'
  - 베이스와 가장 비슷하게 나옴
  - 베이스에선 붙지 않았던 단어가 붙기도(thinksk->thinkskd) 함

## Task2

위 sentence는 제가 임의로 생성한 문장들입니다.

마음에 드시는 문구 가져오셔서 문장이 어떻게 생성되는지 확인해보세요!

영어가 아닌 한국어로 시도해보는 것도 좋겠죠?

수정이 많이 필요(토큰화 등) 할 수 있으나 한번 시도해보시는 것 권장드립니다 :)

위 베이스라인은 어디든 수정하셔도 좋고 조금 더 자연스러운 문장이 나올 수 있게 다양한 시도를 해보세요!

조건 : 문장 3개 이상, 연결성이 있는 문장을 " " 으로 구분하여 ( )에 넣기

In [91]:
my_sentence = ("Life is like a camera Focus on what is important"
            " capture the good times, develop from the negatives"
            " and if things do not work out, take another shot.")

In [92]:
my_word_set = list(set(my_sentence))
my_vocab = {word: idx for idx, word in enumerate(my_word_set)}

In [93]:
print(my_word_set)

[',', 'f', 'l', 'L', 'k', 'r', '.', 'F', 'h', ' ', 'm', 'p', 'n', 'g', 'o', 'w', 'v', 't', 'e', 'u', 'i', 'c', 'd', 'a', 's']


In [94]:
print(my_vocab)

{',': 0, 'f': 1, 'l': 2, 'L': 3, 'k': 4, 'r': 5, '.': 6, 'F': 7, 'h': 8, ' ': 9, 'm': 10, 'p': 11, 'n': 12, 'g': 13, 'o': 14, 'w': 15, 'v': 16, 't': 17, 'e': 18, 'u': 19, 'i': 20, 'c': 21, 'd': 22, 'a': 23, 's': 24}


In [95]:
my_vocab_size = len(my_vocab)
print('문자 집합 크기 : {}'.format(my_vocab_size))    # 총 25가지의 문자가 사용됨!

문자 집합 크기 : 25


In [96]:
# Hyperparameter
my_hidden_size = my_vocab_size
my_sequence_length = 10
my_learning_rate = 0.01

In [97]:
my_x_data = []
my_y_data = []

for i in range(0, len(my_sentence) - my_sequence_length):
  x_str = my_sentence[i:i+my_sequence_length]
  y_str = my_sentence[i+1:i+1+my_sequence_length]
  print(i, x_str, "->", y_str)

  my_x_data.append([my_vocab[c] for c in x_str])
  my_y_data.append([my_vocab[d] for d in y_str])

0 Life is li -> ife is lik
1 ife is lik -> fe is like
2 fe is like -> e is like 
3 e is like  ->  is like a
4  is like a -> is like a 
5 is like a  -> s like a c
6 s like a c ->  like a ca
7  like a ca -> like a cam
8 like a cam -> ike a came
9 ike a came -> ke a camer
10 ke a camer -> e a camera
11 e a camera ->  a camera 
12  a camera  -> a camera F
13 a camera F ->  camera Fo
14  camera Fo -> camera Foc
15 camera Foc -> amera Focu
16 amera Focu -> mera Focus
17 mera Focus -> era Focus 
18 era Focus  -> ra Focus o
19 ra Focus o -> a Focus on
20 a Focus on ->  Focus on 
21  Focus on  -> Focus on w
22 Focus on w -> ocus on wh
23 ocus on wh -> cus on wha
24 cus on wha -> us on what
25 us on what -> s on what 
26 s on what  ->  on what i
27  on what i -> on what is
28 on what is -> n what is 
29 n what is  ->  what is i
30  what is i -> what is im
31 what is im -> hat is imp
32 hat is imp -> at is impo
33 at is impo -> t is impor
34 t is impor ->  is import
35  is import -> is importa
36

In [98]:
my_x_one_hot = [np.eye(my_vocab_size)[i] for i in my_x_data]

my_X = torch.FloatTensor(my_x_one_hot)
my_Y = torch.LongTensor(my_y_data)

In [99]:
print('훈련 데이터의 크기 : {}'.format(my_X.shape))
print('레이블의 크기 : {}'.format(my_Y.shape))

훈련 데이터의 크기 : torch.Size([139, 10, 25])
레이블의 크기 : torch.Size([139, 10])


In [100]:
my_x_data[0]

[3, 20, 1, 18, 9, 20, 24, 9, 2, 20]

In [101]:
my_x_one_hot[0]     # 순서대로 3, 20, 1, 18, ... 에 1이 입력됨

array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

In [102]:
my_y_data[0]

[20, 1, 18, 9, 20, 24, 9, 2, 20, 4]

In [103]:
class my_Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(my_Net, self).__init__()
    self.rnn = torch.nn.RNN(input_size=input_dim, hidden_size=hidden_dim, num_layers=layers, batch_first=True)
    self.fc = torch.nn.Linear(hidden_dim, input_dim)

  def forward(self, x):
    x, _=self.rnn(x)
    x=self.fc(x)
    return x

In [104]:
net = my_Net(my_vocab_size, my_hidden_size, 2)
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), my_learning_rate)
outputs = net(my_X)

In [105]:
for i in range(100):
    optimizer.zero_grad()
    outputs = net(my_X)
    loss = criterion(outputs.view(-1, my_vocab_size), my_Y.view(-1))
    loss.backward()
    optimizer.step()

    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0:
            predict_str += ''.join([my_word_set[t] for t in result])
        else:
            predict_str += my_word_set[result[-1]]

    print(predict_str)

fcfffffffffffffffffffcfffffffffffffcffffffffffffcffffcfLcfffcffffcffffffffffffffcffffffffcfffffcfffffffcffffffffffffcfffcfffLffffffffffcfffffffcffff
fcffnffnffnfnfnlfffffcfnlffnffnffffnffnffnffftffnlffftfntfnfnfffftffffffnfffftffnffffnfffnfffffnfnfnfffcffnfffnffnfetfffnfftntfffttfftfnffffnftnffff
tn     n         t n t     n     t t  t      tt n  t tt t   n           n    t           t                n  t          n  t      t    n    n t     
t                                                                                                                                                   
t                                                                                                                                                   
t                                                                                                                                                   
t                                                                                                         

In [106]:
predict_str

'ife is like a camera Focus on what is important capture the good times, develop from the negatives and if things do not work out, take another shoth'

- 마찬가지로 첫 문자가 학습이 되지 않았지만 거의 완벽한 문장이 생성됨
- 마지막 ife와 shoth만 제외하면 완벽함

In [107]:
# learning_rate를 높여보자
my_hidden_size = my_vocab_size
my_sequence_length = 10
my_learning_rate = 0.05

my_x_data = []
my_y_data = []

for i in range(0, len(my_sentence) - my_sequence_length):
  x_str = my_sentence[i:i+my_sequence_length]
  y_str = my_sentence[i+1:i+1+my_sequence_length]
  print(i, x_str, "->", y_str)

  my_x_data.append([my_vocab[c] for c in x_str])
  my_y_data.append([my_vocab[d] for d in y_str])

my_x_one_hot = [np.eye(my_vocab_size)[i] for i in my_x_data]

my_X = torch.FloatTensor(my_x_one_hot)
my_Y = torch.LongTensor(my_y_data)

class my_Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(my_Net, self).__init__()
    self.rnn = torch.nn.RNN(input_size=input_dim, hidden_size=hidden_dim, num_layers=layers, batch_first=True)
    self.fc = torch.nn.Linear(hidden_dim, input_dim)

  def forward(self, x):
    x, _=self.rnn(x)
    x=self.fc(x)
    return x

net = my_Net(my_vocab_size, my_hidden_size, 2)
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), my_learning_rate)
outputs = net(my_X)

for i in range(100):
    optimizer.zero_grad()
    outputs = net(my_X)
    loss = criterion(outputs.view(-1, my_vocab_size), my_Y.view(-1))
    loss.backward()
    optimizer.step()

    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0:
            predict_str += ''.join([my_word_set[t] for t in result])
        else:
            predict_str += my_word_set[result[-1]]

    print(predict_str)

0 Life is li -> ife is lik
1 ife is lik -> fe is like
2 fe is like -> e is like 
3 e is like  ->  is like a
4  is like a -> is like a 
5 is like a  -> s like a c
6 s like a c ->  like a ca
7  like a ca -> like a cam
8 like a cam -> ike a came
9 ike a came -> ke a camer
10 ke a camer -> e a camera
11 e a camera ->  a camera 
12  a camera  -> a camera F
13 a camera F ->  camera Fo
14  camera Fo -> camera Foc
15 camera Foc -> amera Focu
16 amera Focu -> mera Focus
17 mera Focus -> era Focus 
18 era Focus  -> ra Focus o
19 ra Focus o -> a Focus on
20 a Focus on ->  Focus on 
21  Focus on  -> Focus on w
22 Focus on w -> ocus on wh
23 ocus on wh -> cus on wha
24 cus on wha -> us on what
25 us on what -> s on what 
26 s on what  ->  on what i
27  on what i -> on what is
28 on what is -> n what is 
29 n what is  ->  what is i
30  what is i -> what is im
31 what is im -> hat is imp
32 hat is imp -> at is impo
33 at is impo -> t is impor
34 t is impor ->  is import
35  is import -> is importa
36

In [108]:
predict_str

'ife is like a camera Focus on what is important capture the good times, develop from the negatives and if things do not work out, take another shot.'

- 첫문자 생략만 제외하면 완벽한 문장이 구사됨!
- 마지막 shot 오류 잡아냄